### EXTRAÇÃO DOS DADOS 
Processo de conexão e requisição via <i>API</i>, a requisição foi dividia em dois blocos: <hr>

In [191]:
import pandas as pd
import requests

###### 1.0 - REQUISIÇÃO LISTA DE DEPUTADOS
Requisição dos deputados em mantato na API Dados Abertos

In [192]:
# REQUISIÇÃO VIA API 
url_requisicao_deputados = "https://dadosabertos.camara.leg.br/api/v2/deputados?&ordem=ASC&ordenarPor=nome"
resultado = requests.get(url_requisicao_deputados)
deputados = pd.DataFrame(resultado.json()['dados'])

# PROCESSO DE EXCLUSÃO DE DADOS DESNECESSÁRIOS
lista_exclusao_deputados = ['uri', 'uriPartido', 'urlFoto', 'idLegislatura']
i = 0

while i < len(lista_exclusao_deputados):
    del deputados[lista_exclusao_deputados[i]] 
    i += 1

# TRATAMENTO DOS DADOS
deputados.nome = [str(nome).upper() for nome in deputados.nome]
deputados.email = [str(email).upper() for email in deputados.email]

deputados.columns = ['id','nome','siglaPartido','ufLegislacao','email']

deputados.tail()

,id,nome,siglaPartido,ufLegislacao,email
508,178889,ZÉ CARLOS,PT,MA,DEP.ZECARLOS@CAMARA.LEG.BR
509,204559,ZÉ NETO,PT,BA,DEP.ZENETO@CAMARA.LEG.BR
510,160632,ZÉ SILVA,SOLIDARIEDADE,MG,DEP.ZESILVA@CAMARA.LEG.BR
511,204517,ZÉ VITOR,PL,MG,DEP.ZEVITOR@CAMARA.LEG.BR
512,160592,ZECA DIRCEU,PT,PR,DEP.ZECADIRCEU@CAMARA.LEG.BR


###### 1.1 - REQUISIÇÃO DETALHE DOS DEPUTADOS
Informações adicionais dos deputados

In [193]:
# VARIÁVEIS
id_deputado = deputados['id']
lista_id = []
lista_escolaridade = []
lista_data_nascimento = []
lista_municipio_nascimento = []
lista_sexo = []
lista_estado_nascimento = []

# FUNÇÃO QUE PEGA O ID DO DEPUTADO, FAZ REQUISIÇÃO DA API E SALVA EM UMA VARIÁVEL
def consulta_deputados():
    i = 0
    while i < len(id_deputado):
        conn = requests.get("https://dadosabertos.camara.leg.br/api/v2/deputados/"+str(id_deputado[i]))
        JSON = conn.json()['dados']
        # CONSULTA DADOS 
        lista_id.extend([JSON['id']])
        lista_data_nascimento.extend([JSON['dataNascimento']])
        lista_escolaridade.extend([JSON['escolaridade']])
        lista_municipio_nascimento.extend([JSON['municipioNascimento']])
        lista_sexo.extend([JSON['sexo']])
        lista_estado_nascimento.extend([JSON['ufNascimento']])
        i += 1 

consulta_deputados()

# PROCESSO DE SALVAR OS DADOS EM UM DATAFRAME E RENOMEAR COLUNAS
deputados_detalhe = pd.DataFrame(list(zip(lista_id,lista_escolaridade,lista_data_nascimento,lista_sexo,lista_estado_nascimento, lista_municipio_nascimento)))
deputados_detalhe.columns=['id','escolaridade','dataNascimento','sexo','estadoNascimento','cidadeNascimento']


# TRATAMENTO DOS DADOS
deputados_detalhe.escolaridade = [str(escolaridade).upper() for escolaridade in deputados_detalhe.escolaridade]
deputados_detalhe.cidadeNascimento = [str(cidadeNascimento).upper() for cidadeNascimento in deputados_detalhe.cidadeNascimento]

deputados_detalhe.head()

,id,escolaridade,dataNascimento,sexo,estadoNascimento,cidadeNascimento
0,204554,SUPERIOR INCOMPLETO,1965-02-13,M,BA,SALVADOR
1,204521,SUPERIOR,1966-11-06,M,SP,SÃO PAULO
2,204379,SUPERIOR,1983-09-28,M,AP,MACAPÁ
3,204560,SUPERIOR,1981-02-02,M,BA,SALVADOR
4,204528,DOUTORADO,1969-03-06,F,SP,SÃO PAULO


###### 1.2 - JOIN DAS TABELAS DEPUTADOS E DEPUTADOS DETALHE

In [195]:
deputados_join = pd.merge(deputados, deputados_detalhe, how='left',on='id')
deputados_join.head()

,id,nome,siglaPartido,ufLegislacao,email,escolaridade,dataNascimento,sexo,estadoNascimento,cidadeNascimento
0,204554,ABÍLIO SANTANA,PL,BA,DEP.ABILIOSANTANA@CAMARA.LEG.BR,SUPERIOR INCOMPLETO,1965-02-13,M,BA,SALVADOR
1,204521,ABOU ANNI,PSL,SP,DEP.ABOUANNI@CAMARA.LEG.BR,SUPERIOR,1966-11-06,M,SP,SÃO PAULO
2,204379,ACÁCIO FAVACHO,PROS,AP,DEP.ACACIOFAVACHO@CAMARA.LEG.BR,SUPERIOR,1983-09-28,M,AP,MACAPÁ
3,204560,ADOLFO VIANA,PSDB,BA,DEP.ADOLFOVIANA@CAMARA.LEG.BR,SUPERIOR,1981-02-02,M,BA,SALVADOR
4,204528,ADRIANA VENTURA,NOVO,SP,DEP.ADRIANAVENTURA@CAMARA.LEG.BR,DOUTORADO,1969-03-06,F,SP,SÃO PAULO


###### 1.3 - CRIANDO BASE DE DADOS EM CSV

In [198]:
deputados_join.to_csv('db/db_lista_de_deputados.csv', encoding='utf-8', index=False)

###### 2 - REQUISIÇÃO LISTA DE GASTOS

In [ ]:
gastos = []
num_pagina = list(range(1,19,1))
t = 0 
while t < len(num_pagina):
    for id in deputados.id:
        url_despesa = "https://dadosabertos.camara.leg.br/api/v2/deputados/"
        url_despesa = url_despesa +str(id)+"/despesas?ano=2019&ano=2020&itens=999999999&pagina="+str(num_pagina[t])
        resposta = requests.get(url_despesa)
        gasto = pd.DataFrame(resposta.json()['dados'])
        gasto['id'] = id
        gastos.append(gasto)
    print('PÁGINA: ', num_pagina[t])
    t +=1
    
total_gastos = pd.concat(gastos)
lista_exclusao_gastos = ['numRessarcimento','dataDocumento','codDocumento', 'codTipoDocumento','numDocumento', 'urlDocumento','codLote', 'parcela']

i = 0
while i < len(lista_exclusao_gastos):
    del total_gastos[lista_exclusao_gastos[i]] 
    i += 1
    
print('CONSULTA FINALIZADA')

######  2.1 TRATAMENTO DOS DADOS
Ajuste e padronização nas nomenclaturas.

In [184]:
# PADRONIZAÇÃO DE VARIÁVEIS PARA UM MESMO NOME NA COLUNA FORNECEDOR:
total_gastos.nomeFornecedor = [str(fornecedor).upper().replace("S.A.", "S.A").replace("S/A","S.A").replace("LTDA-ME","LTDA") for fornecedor in total_gastos.nomeFornecedor]

# PADRONIZAÇÃO DE ESCRITA PARA CAIXA ALTA:
total_gastos.tipoDespesa = [str(despesas).upper() for despesas in total_gastos.tipoDespesa]

# AJUSTANDO O MÊS E ANO:
mes_incorreto = ['1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0','10.0','11.0','12.0']
mes_correto = list(range(1,13,1))

i = 0 
while i < len(mes_incorreto):
    total_gastos.mes = [str(mes_list).replace(str(mes_incorreto[i]), str(mes_correto[i])) for mes_list in total_gastos.mes]
    i +=1
total_gastos.ano = [str(ano_list).replace("2020.0", "2020").replace("2019.0", "2019") for ano_list in total_gastos.ano]

total_gastos.head()

,ano,mes,tipoDespesa,tipoDocumento,valorDocumento,nomeFornecedor,cnpjCpfFornecedor,valorLiquido,valorGlosa,id
0,2019,3,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,154.95,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,154.95,0.00,204554
1,2019,5,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,343.48,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,335.80,7.68,204554
2,2019,6,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,312.27,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,305.26,7.01,204554
3,2019,7,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,191.56,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,184.53,7.03,204554
4,2019,8,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,138.61,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,131.90,6.71,204554


###### 2.2 - CRIANDO BASE DE DADOS EM CSV

In [189]:
total_gastos.to_csv('db/db_lista_total_gastos.csv', encoding='utf-8', index=False)